In [1]:
%matplotlib inline

In [2]:
import jax
from jax.config import config
config.update("jax_enable_x64", True)

/tmp/ipykernel_1884605/2441880800.py:2: DeprecationWarning: Accessing jax.config via the jax.config submodule is deprecated.
  from jax.config import config


In [3]:
import scanpy as sc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import moscot
from moscot.problems.time import TemporalProblem
import moscot.plotting as mpl
import pandas as pd
import os
from scipy.sparse import csr_matrix

import mplscience

mplscience.available_styles()
mplscience.set_style(reset_current=True)
plt.rcParams['legend.scatterpoints'] = 1 

['default', 'despine']


In [4]:
output_dir = "/lustre/groups/ml01/workspace/moscot_paper/pancreas_revision/plots/driver_features"

In [5]:
tp0 = TemporalProblem.load("/lustre/groups/ml01/workspace/moscot_paper/pancreas_revision/plots/OT_encodrine_analysis/TemporalProblem.pkl")

In [6]:
tp0.adata.X = tp0.adata.layers["raw_counts"].copy()

In [7]:
sc.pp.normalize_total(tp0.adata)
sc.pp.log1p(tp0.adata)

# Delta

In [8]:
tp0.pull(14.5, 15.5, data="cell_type", subset="Delta",  key_added = "Delta_pull_early", normalize=False)
tp0.pull(15.5, 16.5, data="cell_type", subset="Delta",  key_added = "Delta_pull_late", normalize=False)
tp0.adata.obs["Delta_pull"] = tp0.adata.obs["Delta_pull_early"] + tp0.adata.obs["Delta_pull_late"]

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning:

In [9]:
delta_drivers = tp0.compute_feature_correlation(obs_key="Delta_pull", annotation={"time": [14.5, 15.5]})

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [10]:
delta_drivers

,Delta_pull_corr,Delta_pull_pval,Delta_pull_qval,Delta_pull_ci_low,Delta_pull_ci_high
Hhex,0.661332,0.000000e+00,0.000000e+00,0.645737,0.676377
Sst,0.637607,0.000000e+00,0.000000e+00,0.621168,0.653486
Mef2c,0.448428,1.389060e-264,9.308093e-261,0.426413,0.469913
Dscam,0.436931,2.888837e-249,1.451857e-245,0.414642,0.458696
Ptprz1,0.422747,3.017971e-231,1.213406e-227,0.400131,0.444848
...,...,...,...,...,...
Igkv6-32,NaN,NaN,NaN,NaN,NaN
Hgf,NaN,NaN,NaN,NaN,NaN
Fgf5,NaN,NaN,NaN,NaN,NaN
Cyp26a1,NaN,NaN,NaN,NaN,NaN


In [11]:
delta_drivers.to_csv(os.path.join(output_dir, "driver_delta_genes.csv"))

In [12]:
delta_drivers_tfs = tp0.compute_feature_correlation(obs_key="Delta_pull", annotation={"time": [14.5, 15.5]}, features="mouse")

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [13]:
delta_drivers_tfs.head(10)

,Delta_pull_corr,Delta_pull_pval,Delta_pull_qval,Delta_pull_ci_low,Delta_pull_ci_high
Hhex,0.661332,0.000000e+00,0.000000e+00,0.645737,0.676377
Mef2c,0.448428,1.389060e-264,1.048741e-261,0.426413,0.469913
Arg1,0.365217,3.094459e-167,1.557544e-164,0.341391,0.388574
Isl1,0.244520,3.597763e-72,1.358156e-69,0.218756,0.269943
Ikzf2,0.186334,5.867192e-42,1.265637e-39,0.159925,0.212477
Luzp2,0.169645,6.143412e-35,1.159569e-32,0.143086,0.195961
Meis2,0.165809,2.008001e-33,3.368980e-31,0.139217,0.192162
Zfp521,0.157627,2.568306e-30,3.525583e-28,0.130969,0.184057
Zbtb20,0.147429,1.124930e-26,1.415537e-24,0.120693,0.173951
Rora,0.143221,3.016324e-25,3.503576e-23,0.116455,0.169779


In [14]:
delta_drivers_tfs.to_csv(os.path.join(output_dir, "driver_delta_tfs.csv"))

# Epsilon

In [15]:
tp0.pull(14.5, 15.5, data="cell_type", subset="Epsilon",  key_added = "Epsilon_pull_early", normalize=False)
tp0.pull(15.5, 16.5, data="cell_type", subset="Epsilon",  key_added = "Epsilon_pull_late", normalize=False)
tp0.adata.obs["Epsilon_pull"] = tp0.adata.obs["Epsilon_pull_early"] + tp0.adata.obs["Epsilon_pull_late"]

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning:

In [16]:
eps_drivers = tp0.compute_feature_correlation(obs_key="Epsilon_pull", annotation={"time": [14.5, 15.5]})

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [17]:
eps_drivers.to_csv(os.path.join(output_dir, "driver_eps_genes.csv"))

In [18]:
eps_drivers_tfs = tp0.compute_feature_correlation(obs_key="Epsilon_pull", features="mouse", annotation={"time": [14.5, 15.5]})

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [19]:
eps_drivers_tfs.to_csv(os.path.join(output_dir, "driver_eps_tfs.csv"))

# Epsilon progenitors

In [20]:
tp0.pull(14.5, 15.5, data="cell_type", subset="Eps. progenitors",  key_added = "Eps_progenitors_pull_early", normalize=False)
tp0.pull(15.5, 16.5, data="cell_type", subset="Eps. progenitors",  key_added = "Eps_progenitors_pull_late", normalize=False)
tp0.adata.obs["Eps_progenitors_pull"] = tp0.adata.obs["Eps_progenitors_pull_early"] + tp0.adata.obs["Eps_progenitors_pull_late"]

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning:

In [21]:
eps_prog_drivers = tp0.compute_feature_correlation(obs_key="Eps_progenitors_pull", annotation={"time": [14.5, 15.5]})

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [22]:
eps_prog_drivers

,Eps_progenitors_pull_corr,Eps_progenitors_pull_pval,Eps_progenitors_pull_qval,Eps_progenitors_pull_ci_low,Eps_progenitors_pull_ci_high
Cdkn1a,0.239425,3.734786e-69,6.085183e-65,0.213597,0.264919
Ppp1r14a,0.239066,6.054004e-69,6.085183e-65,0.213233,0.264564
Btbd17,0.227906,1.326704e-62,8.890242e-59,0.201938,0.253553
Tbc1d9,0.223067,5.816654e-60,2.362201e-56,0.197043,0.248776
Kif26a,0.223059,5.875246e-60,2.362201e-56,0.197035,0.248768
...,...,...,...,...,...
Igkv6-32,NaN,NaN,NaN,NaN,NaN
Hgf,NaN,NaN,NaN,NaN,NaN
Fgf5,NaN,NaN,NaN,NaN,NaN
Cyp26a1,NaN,NaN,NaN,NaN,NaN


In [23]:
eps_prog_drivers.to_csv(os.path.join(output_dir, "driver_eps_prog_genes.csv"))

In [24]:
eps_prog_drivers_tf = tp0.compute_feature_correlation(obs_key="Eps_progenitors_pull", features="mouse", annotation={"time": [14.5, 15.5]})

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [25]:
eps_prog_drivers_tf.head(10)

,Eps_progenitors_pull_corr,Eps_progenitors_pull_pval,Eps_progenitors_pull_qval,Eps_progenitors_pull_ci_low,Eps_progenitors_pull_ci_high
Gadd45a,0.185422,1.480132e-41,1.117500e-38,0.159005,0.211575
Neurod2,0.184934,2.424185e-41,1.220173e-38,0.158512,0.211092
Rps4x,0.156165,8.856936e-30,2.674795e-27,0.129495,0.182609
Setbp1,0.142850,4.013451e-25,7.447096e-23,0.116081,0.169411
Nkx2-2,0.142700,4.501829e-25,7.447096e-23,0.115930,0.169263
Neurog3,0.142581,4.931852e-25,7.447096e-23,0.115810,0.169145
Drap1,0.140736,2.009380e-24,2.758331e-22,0.113952,0.167315
Sox4,0.135847,7.577794e-23,8.801899e-21,0.109030,0.162466
Tead2,0.130612,3.191282e-21,3.212557e-19,0.103761,0.157273
Hes6,0.127552,2.649001e-20,2.352936e-18,0.100682,0.154237


In [26]:
eps_prog_drivers_tf.to_csv(os.path.join(output_dir, "driver_eps_prog_tfs.csv"))

# Fev+ Delta

In [27]:
tp0.pull(14.5, 15.5, data="cell_type", subset="Fev+ Delta",  key_added = "Fev_delta_pull_early", normalize=False)
tp0.pull(15.5, 16.5, data="cell_type", subset="Fev+ Delta",  key_added = "Fev_delta_pull_late", normalize=False)
tp0.adata.obs["Fev_delta_pull"] = tp0.adata.obs["Fev_delta_pull_early"] + tp0.adata.obs["Fev_delta_pull_late"]

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning:

In [28]:
fev_delta_drivers =  tp0.compute_feature_correlation(obs_key="Fev_delta_pull", annotation={"time": [14.5, 15.5]})

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [29]:
fev_delta_drivers

,Fev_delta_pull_corr,Fev_delta_pull_pval,Fev_delta_pull_qval,Fev_delta_pull_ci_low,Fev_delta_pull_ci_high
Cdkn1a,0.330532,6.228073e-135,1.252029e-130,0.306065,0.354562
Mboat4,0.308716,8.754341e-117,8.799426e-113,0.283882,0.333137
Rgs17,0.260188,6.508473e-82,4.361328e-78,0.234630,0.285387
Nefm,0.254982,1.348873e-78,6.779099e-75,0.229353,0.280257
Gm38655,0.233234,1.375951e-65,5.532148e-62,0.207330,0.258811
...,...,...,...,...,...
Igkv6-32,NaN,NaN,NaN,NaN,NaN
Hgf,NaN,NaN,NaN,NaN,NaN
Fgf5,NaN,NaN,NaN,NaN,NaN
Cyp26a1,NaN,NaN,NaN,NaN,NaN


In [30]:
fev_delta_drivers.to_csv(os.path.join(output_dir, "drivers_fev_delta_genes.csv"))

In [31]:
fev_delta_drivers_tfs = tp0.compute_feature_correlation(obs_key="Fev_delta_pull",features="mouse", annotation={"time": [14.5, 15.5]})

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [32]:
fev_delta_drivers_tfs.head()

,Fev_delta_pull_corr,Fev_delta_pull_pval,Fev_delta_pull_qval,Fev_delta_pull_ci_low,Fev_delta_pull_ci_high
Isl1,0.174970,4.221143e-37,6.373925e-34,0.148457,0.201232
Neurod2,0.165568,2.491138e-33,1.880809e-30,0.138974,0.191924
Prox1,0.158984,8.050395e-31,3.039024e-28,0.132336,0.185402
Peg3,0.152911,1.333852e-28,4.028234e-26,0.126216,0.179385
Jund,0.129758,5.792914e-21,1.249614e-18,0.102901,0.156425


In [33]:
fev_delta_drivers_tfs.to_csv(os.path.join(output_dir, "drivers_fev_delta_tfs.csv"))

# Alpha

In [34]:
tp0.pull(14.5, 15.5, data="cell_type", subset="Alpha",  key_added = "Alpha_pull_early", normalize=False)
tp0.pull(15.5, 16.5, data="cell_type", subset="Alpha",  key_added = "Alpha_pull_late", normalize=False)
tp0.adata.obs["Alpha_pull"] = tp0.adata.obs["Alpha_pull_early"] + tp0.adata.obs["Alpha_pull_late"]

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning:

In [35]:
alpha_drivers = tp0.compute_feature_correlation(obs_key="Alpha_pull", annotation={"time": [14.5, 15.5]})

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [36]:
alpha_drivers 

,Alpha_pull_corr,Alpha_pull_pval,Alpha_pull_qval,Alpha_pull_ci_low,Alpha_pull_ci_high
Cltrn,0.642486,0.0,0.0,0.626217,0.658195
Gcg,0.634646,0.0,0.0,0.618104,0.650627
Peg10,0.604871,0.0,0.0,0.587321,0.621852
Wnk3,0.596221,0.0,0.0,0.578387,0.613485
Mctp2,0.595638,0.0,0.0,0.577786,0.612921
...,...,...,...,...,...
Igkv6-32,NaN,NaN,NaN,NaN,NaN
Hgf,NaN,NaN,NaN,NaN,NaN
Fgf5,NaN,NaN,NaN,NaN,NaN
Cyp26a1,NaN,NaN,NaN,NaN,NaN


In [37]:
alpha_drivers.to_csv(os.path.join(output_dir, "drivers_alpha_genes.csv"))

In [38]:
alpha_drivers_tfs = tp0.compute_feature_correlation(obs_key="Alpha_pull", annotation={"time": [14.5, 15.5]}, features="mouse")

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [39]:
alpha_drivers_tfs

,Alpha_pull_corr,Alpha_pull_pval,Alpha_pull_qval,Alpha_pull_ci_low,Alpha_pull_ci_high
Mctp2,0.595638,0.0,0.0,0.577786,0.612921
Irx2,0.567377,0.0,0.0,0.548630,0.585554
Irx1,0.541528,0.0,0.0,0.522003,0.560487
Smarca1,0.539894,0.0,0.0,0.520320,0.558900
Pou6f2,0.499082,0.0,0.0,0.478360,0.519248
...,...,...,...,...,...
Twist2,NaN,NaN,NaN,NaN,NaN
Tal1,NaN,NaN,NaN,NaN,NaN
Irx5,NaN,NaN,NaN,NaN,NaN
Nr5a1,NaN,NaN,NaN,NaN,NaN


In [40]:
alpha_drivers_tfs.to_csv(os.path.join(output_dir, "drivers_alpha_tfs.csv"))

# Beta

In [41]:
tp0.pull(14.5, 15.5, data="cell_type", subset="Beta",  key_added = "Beta_pull_early", normalize=False)
tp0.pull(15.5, 16.5, data="cell_type", subset="Beta",  key_added = "Beta_pull_late", normalize=False)
tp0.adata.obs["Beta_pull"] = tp0.adata.obs["Beta_pull_early"] + tp0.adata.obs["Beta_pull_late"]

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning:

In [42]:
beta_drivers = tp0.compute_feature_correlation(obs_key="Beta_pull", annotation={"time": [14.5, 15.5]})

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [43]:
beta_drivers

,Beta_pull_corr,Beta_pull_pval,Beta_pull_qval,Beta_pull_ci_low,Beta_pull_ci_high
Sytl4,0.751571,0.0,0.0,0.739479,0.763178
Phactr1,0.747111,0.0,0.0,0.734834,0.758897
Nnat,0.743702,0.0,0.0,0.731285,0.755625
Sntg1,0.739035,0.0,0.0,0.726428,0.751145
Ins2,0.730049,0.0,0.0,0.717079,0.742514
...,...,...,...,...,...
Igkv6-32,NaN,NaN,NaN,NaN,NaN
Hgf,NaN,NaN,NaN,NaN,NaN
Fgf5,NaN,NaN,NaN,NaN,NaN
Cyp26a1,NaN,NaN,NaN,NaN,NaN


In [44]:
beta_drivers.to_csv(os.path.join(output_dir, "drivers_beta_genes.csv"))

In [45]:
beta_drivers_tfs = tp0.compute_feature_correlation(obs_key="Beta_pull", annotation={"time": [14.5, 15.5]}, features="mouse")

/home/icb/dominik.klein/mambaforge/envs/moscot_up/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [46]:
beta_drivers_tfs

,Beta_pull_corr,Beta_pull_pval,Beta_pull_qval,Beta_pull_ci_low,Beta_pull_ci_high
Mafa,0.668961,0.000000e+00,0.000000e+00,0.653643,0.683731
Hspa5,0.553791,0.000000e+00,0.000000e+00,0.534630,0.572383
Rora,0.543927,0.000000e+00,0.000000e+00,0.524472,0.562815
Mlxipl,0.470295,1.382902e-295,5.220454e-293,0.448821,0.491227
Mafb,0.431096,9.739473e-242,2.941321e-239,0.408671,0.453000
...,...,...,...,...,...
Twist2,NaN,NaN,NaN,NaN,NaN
Tal1,NaN,NaN,NaN,NaN,NaN
Irx5,NaN,NaN,NaN,NaN,NaN
Nr5a1,NaN,NaN,NaN,NaN,NaN


In [47]:
beta_drivers_tfs.to_csv(os.path.join(output_dir, "drivers_beta_tfs.csv"))

In [48]:
tp0.adata.write(os.path.join(output_dir, "adata_driver.h5ad"))